<a href="https://colab.research.google.com/github/Madhukar-Palakaveeti/deep_learning/blob/main/nninitial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
a = np.array([1,2,3,4])
b = np.array([[1,2,3,4],[5,6,7,8]])
c = np.array([[[1,2,3,4],[5,6,7,8]],[[2,3,4,5],[4,5,6,7]]])
print(a.shape)
print(b.shape)
print(c.shape)

(4,)
(2, 4)
(2, 2, 4)


In [2]:
z = np.zeros((3,2,4))
print(z)
print(z.shape)

[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
(3, 2, 4)


In [3]:
print(np.dot(b,a))
print(np.dot(c,a))

[30 70]
[[30 70]
 [40 60]]


In [4]:
I = np.identity(4)
print(I)
print(c)
print(I.shape)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[[1 2 3 4]
  [5 6 7 8]]

 [[2 3 4 5]
  [4 5 6 7]]]
(4, 4)


In [5]:
print(np.dot(c,I))

[[[1. 2. 3. 4.]
  [5. 6. 7. 8.]]

 [[2. 3. 4. 5.]
  [4. 5. 6. 7.]]]


In [6]:
inputs = [1,2,3,2.5]
weights = [[0.2,0.8,-0.5,1.0],[0.5,-0.91,0.26,-0.5],[-0.26,-0.27,0.17,0.87]]
biases = [2,3,0.5]
print(np.dot(weights,inputs)+biases)

[4.8   1.21  2.385]


In [7]:
X = np.array([[1,2,3,2.5],[2.0,5.0,-1.0,2.0],[-1.5,2.7,3.3,-0.8]])
print(X)

[[ 1.   2.   3.   2.5]
 [ 2.   5.  -1.   2. ]
 [-1.5  2.7  3.3 -0.8]]


In [8]:
import numpy as np
np.random.seed(0)


In [17]:
class Layer_Dense:
  def __init__(self,n_inputs,n_neurons):
    self.weights = 0.01 * np.random.randn(n_inputs,n_neurons)
    self.biases = np.zeros((1,n_neurons))

  def forward(self,inputs):
    self.output = np.dot(inputs, self.weights)+ self.biases

class Activation_ReLU:
  def forward(self,inputs):
    self.output = np.maximum(0,inputs)

class Activation_FakeSoftmax:
  def forward(self,inputs):
    exp_values = np.exp(inputs)
    probabilities = exp_values/np.sum(exp_values,axis=1,keepdims=True)
    self.output = probabilities

class Activation_Softmax:
  def forward(self,inputs):
    exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
    probabilities = exp_values/np.sum(exp_values,axis=1,keepdims=True)
    self.output = probabilities

class Loss:
  def calculate(self,output,y):
    sample_losses = self.forward(output,y)
    data_loss = np.mean(sample_losses)
    return data_loss

class Loss_Categoricalcrossentropy(Loss): #Need to reimplement the whole loss function for just one-hot encoded true labels
  def forward(self, y_pred, y_true):
    samples = len(y_pred)
    y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
      negative_log_likelihoods = -np.log(correct_confidences)
    elif len(y_true.shape) == 2:
      negative_log_likelihoods = -np.sum(y_true*np.log(y_pred_clipped)) #this is just a quick fix


    return negative_log_likelihoods




In [10]:
!pip install nnfs

In [11]:
import nnfs
nnfs.init()
from nnfs.datasets import spiral_data

In [12]:
X, y = spiral_data(samples=100, classes=3)

In [13]:
fake_layer = Layer_Dense(2,5)
fake_layer.forward(X)
# activation1 = Activation_ReLU()
# activation1.forward(layer1.output)
# print(activation1.output[:5])
activation2 = Activation_FakeSoftmax()
activation2.forward(fake_layer.output)
print("Fake Softmax")
print(activation2.output[:5])

activation3 = Activation_Softmax()
activation3.forward(fake_layer.output)
print("Softmax")
print(activation3.output[:5])


Fake Softmax
[[0.2        0.2        0.2        0.2        0.2       ]
 [0.19999309 0.199994   0.1999831  0.2000191  0.20001073]
 [0.19996709 0.20001628 0.19997013 0.20001917 0.2000273 ]
 [0.19991961 0.20008954 0.19998294 0.1999674  0.20004053]
 [0.19989496 0.20011176 0.19997184 0.19996573 0.20005569]]
Softmax
[[0.2        0.2        0.2        0.2        0.2       ]
 [0.1999931  0.199994   0.1999831  0.2000191  0.2000107 ]
 [0.19996712 0.20001629 0.19997014 0.20001918 0.20002732]
 [0.1999196  0.20008956 0.19998293 0.1999674  0.20004052]
 [0.19989496 0.20011178 0.19997185 0.19996572 0.20005567]]


In [26]:
# @title Implementing the neural network with two layers and two activation functions
layer1 = Layer_Dense(2,5)
activation1 = Activation_ReLU()

layer2 = Layer_Dense(5,3)
activation2 = Activation_Softmax()

layer1.forward(X)
activation1.forward(layer1.output)
print(f"Activation1 : {activation1.output.shape}")
print("Activation 1 Transpose")
print(activation1.output.T.shape)
layer2.forward(activation1.output)
activation2.forward(layer2.output)
print(f"Activation2 : {activation2.output.shape}")

loss_function = Loss_Categoricalcrossentropy()
loss = loss_function.forward(activation2.output,y)
#print(f"Loss:{loss.shape}")
y_dash = np.array([[0,0,1],[0,0,1],[0,0,1],[0,0,1],[0,0,1]])
y_pred = activation2.output
dz2 = y_pred[-1:-6:-1] - y_dash
dw2 = np.dot(activation1.output[-1:-6:-1],dz2)
#print(f"dz2 : {dz2}")
#print(f"dw2 : {dw2}")

#print(f"bias : {layer1.biases}")

print(y_pred[-1:-6:-1])
#print(y_pred[range(len(y_pred)),y][-1:-6:-1])
#y_pred = np.argmax(activation2.output,axis=1)
#accuracy = np.mean(y_pred == y)
#print(np.clip(y_pred,0.0000001, -0.0000001 ))
#print(y[-1:-6:-1])
#print(f"Accuracy:{accuracy*100}")

Activation1 : (300, 5)
Activation 1 Transpose
(5, 300)
Activation2 : (300, 3)
[[0.33324292 0.33339962 0.33335742]
 [0.33324707 0.3334035  0.3333494 ]
 [0.33324376 0.3334005  0.33335584]
 [0.33324763 0.33340234 0.33334997]
 [0.33324632 0.33339745 0.33335623]]


In [19]:
# @title LOSS
#categorical_crossentropy for multiclass classification
# mean_squared_error for regression
import math
target_output = [1,0,0]
y_pred = [0.7,0.2,0.1]
loss = -(math.log(y_pred[0]))
print(loss)

0.35667494393873245


In [20]:
print(X[:5], y[:5])

[[0.         0.        ]
 [0.00299556 0.00964661]
 [0.01288097 0.01556285]
 [0.02997479 0.0044481 ]
 [0.03931246 0.00932828]] [0 0 0 0 0]


In [21]:
X.size


600

In [22]:
X[:5]

array([[0.        , 0.        ],
       [0.00299556, 0.00964661],
       [0.01288097, 0.01556285],
       [0.02997479, 0.0044481 ],
       [0.03931246, 0.00932828]], dtype=float32)

In [23]:
(1e-7)

1e-07

In [24]:
x_train = X[-1:-6:-1]
print(x_train)
labels = [0,1,1,0,1]
print(x_train[range(5), labels])

[[-0.9427888   0.33339068]
 [-0.9793868  -0.14387996]
 [-0.95069844  0.23701589]
 [-0.9649202  -0.09613083]
 [-0.91110104  0.3011964 ]]
[-0.9427888  -0.14387996  0.23701589 -0.9649202   0.3011964 ]


In [25]:
print(X.shape)

(300, 2)


In [27]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], d

In [36]:
# @title One-hot encoding the labels
y_true = np.zeros((len(y), len(np.unique(y))))
y_true[np.arange(y.size), y] = 1
y_true


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0